In [1]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime
from pathlib import Path
from datetime import date, timedelta
import pyarrow.parquet as pq

server = '10.40.16.19'
database = 'MSIDo'
username = 'login_fadhil'
password = 'RTvT9XBtcbjO'
TrustServerCertificate = 'yes'
engine = create_engine(f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=/opt/microsoft/msodbcsql18/lib64/libmsodbcsql-18.3.so.2.1&TrustServerCertificate={TrustServerCertificate}")

today = date.today()
yesterday = today + timedelta(-1)

def fetching_data(yesterday: str, engine) -> pd.DataFrame:
    query = (f"""
         SELECT 
        FORMAT(o.created_at AT TIME ZONE 'UTC' AT TIME ZONE 'SE Asia Standard Time','yyyy-MM-dd') AS tanggal,
        o.created_at as order_date,
        o.id as order_id,
        o.invoice_code,
	    o.total,
        o.total_final,
        o.total_dpp,
        o.total_ppn,
	    o.total_voucher,
        o.total_discount,
        o.total_golden_promo,
        opd.subtotal,
        opd.dpp,
        opd.golden_promo,
        opd.ppn,
        opd.discount,
        u.id as user_id,
        CONCAT(up.first_name ,' ', up.last_name) as customer,
        TRIM(UPPER(u.referral_by)) AS referral_by,
        p.id as product_id,
        p.name as product_name,
        p.sku as product_sku,
        pb.id as productbrand_id,
        pb.name as productbrand_name,
        d.id as depo_id,
        d.name as depo_name,
        prin.id as principal_id,
        prin.name as principal_name,
        prin.code as principal_code,
        dist.id  as distributor_id,
        dist.name as distributor_name
    from orders o
        left join order_productdetail opd
        on o.id = opd.order_id
        left join users u
        on u.id = o.user_id
        left join users_profiles up
        on u.id = up.user_id
        left join products_details pd
        on pd.id = opd.products_detail_id 
        left join products p
        on p.id = pd.product_id 
        left join products_brands pb
        on pb.id = p.products_brand_id
        LEFT join depo d
        on d.id = o.depo_id
        left join principals prin
        on prin.id = pb.principal_id
        left join distributors dist
        on dist.id = prin.distributor_id

    WHERE FORMAT(o.created_at AT TIME ZONE 'UTC' AT TIME ZONE 'SE Asia Standard Time','yyyy-MM-dd') BETWEEN '2024-01-01' AND '2024-04-03'
    ORDER BY o.created_at ASC;""")
    df = pd.read_sql(query, engine)
    df['tanggal'] = pd.to_datetime(df['tanggal']).dt.date
    print(f"fetching data :\n {df}")
    print(f"fetching data :\n {df.dtypes}")
    return df
def write_parquet(df: pd.DataFrame, dataset_file: str) -> Path:
    path = Path(f"/home/kantor/ingest/replicate/barkul_report/data_parquet/{dataset_file}.parquet")
    df.to_parquet(path, compression="gzip")
    return path
pd.io.parquet.get_engine('auto')

def ingest_order():
    """get data """
    today = date.today()

    yesterday = today + timedelta(-1)
#yesterday_7 = yesterday + timedelta(-6)
    df = fetching_data(str(yesterday),engine)

    """write to parquet"""
    dataset_file = f"order_{yesterday}"
    write_parquet(df, dataset_file)

ingest_order()


fetching data :
            tanggal                       order_date  order_id  \
0       2024-01-01 2023-12-31 17:05:42.398000+00:00   1109258   
1       2024-01-01 2023-12-31 21:35:52.134000+00:00   1109259   
2       2024-01-01 2024-01-01 00:10:09.687000+00:00   1109260   
3       2024-01-01 2024-01-01 00:31:06.316000+00:00   1109261   
4       2024-01-01 2024-01-01 00:39:30.012000+00:00   1109262   
...            ...                              ...       ...   
492704  2024-04-03 2024-04-03 16:56:11.769000+00:00   1161017   
492705  2024-04-03 2024-04-03 16:56:11.769000+00:00   1161017   
492706  2024-04-03 2024-04-03 16:56:11.769000+00:00   1161017   
492707  2024-04-03 2024-04-03 16:56:11.769000+00:00   1161017   
492708  2024-04-03 2024-04-03 16:56:11.769000+00:00   1161017   

            invoice_code      total  total_final   total_dpp  total_ppn  \
0       MS01242401010441  2978364.0   2972364.00  2683210.81  295153.19   
1       MS01102401010441   700726.0    700726.00   6